In [1]:
import pandas
URL = 'Health_care_Dataset_for_probelm.xlsx'
df_training = pandas.read_excel(URL,sheetname=1)
df_testing = pandas.read_excel(URL, sheetname=2)

In [2]:
df_train = df_training[:]
df_test = df_testing[:]

In [3]:
Y_train = df_train['Lung_Cancer'].values
del df_train['Lung_Cancer']
del df_test['Lung_Cancer']
del df_train['Patient_ID']
del df_test['Patient_ID']

In [31]:
import numpy as np
for i in list(df_train.columns.values):
    if len(np.unique(df_train[i].values)) < 5:
        df_train[i] = df_train[i].astype(str)
import numpy as np
for i in list(df_test.columns.values):
    if len(np.unique(df_test[i].values)) < 5:
        df_test[i] = df_test[i].astype(str)


C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#ENCODING
X_train = df_train.to_dict('records')
X_test = df_test.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_train)
X_te.extend(X_test)
X_total = X_tr + X_te

In [5]:
X_te

[{u'Disease1': 0.0,
  u'Disease1Treat': 0.0,
  u'Disease2': 0.0,
  u'Disease2Times': 0.0,
  u'Disease3': 0.33333333300000001,
  u'Disease3Times': 0.14999999999999999,
  u'Disease4': 0.0,
  u'Disease4Treat': 0.0,
  u'Disease5': 0.0,
  u'Disease5Treat': 0.0,
  u'Disease6': 0.33333333300000001,
  u'Disease6Treat': 1.0,
  u'Disease7': 0.0,
  u'DiseaseHis1': 0.0,
  u'DiseaseHis1Times': 0.0,
  u'DiseaseHis2': 0.0,
  u'DiseaseHis2Times': 0.0,
  u'DiseaseHis3': 0.0,
  u'DiseaseHis3Times': 0.0,
  u'DiseaseHis4': 0.0,
  u'DiseaseHis5': 0.0,
  u'DiseaseHis6': 0.0,
  u'DiseaseHis7': 0.0,
  u'DiseaseStage1': 0.80000000000000004,
  u'DiseaseStage2': 0.71428571399999996,
  u'Factor1': 0.0,
  u'Factor2': 0.38714384969999999,
  u'Factor3': 0.53688760830000004,
  u'Factor4': 0.2909669949,
  u'Factor5': 0.0,
  u'Factor6': 0.78099173550000001,
  u'LungFunct1': 0.40269886380000003,
  u'LungFunct10': 0.0,
  u'LungFunct11': 0.15788751750000002,
  u'LungFunct12': 0.69893617050000001,
  u'LungFunct13': 0.17489

In [33]:
#One Hot Encoding 
from sklearn.feature_extraction import DictVectorizer 
enc = DictVectorizer(sparse = True)
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train =X_encoded_total[:len(X_tr)]
X_encoded_test =X_encoded_total[len(X_tr):]

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
estimator = GradientBoostingClassifier() #Using a basic GBM for feature reduction

from sklearn.feature_selection import SelectFromModel
estimator.fit(X_encoded_train.toarray(),Y_train)
model = SelectFromModel(estimator, prefit=True)
X_encoded_train_new = model.transform(X_encoded_train.toarray())
X_encoded_test_new = model.transform(X_encoded_test.toarray())
X_train_new = model.transform(X_train.toarray())
X_test_new = model.transform(X_test.toarray())

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
classifier  = GradientBoostingClassifier(n_estimators=232,loss='exponential',learning_rate=0.30000000000000004,max_depth=1) 
classifier.fit(X_encoded_train_new,Y_train)

GradientBoostingClassifier(init=None, learning_rate=0.3, loss='exponential',
              max_depth=1, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=232,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [17]:

from sklearn.cross_validation import KFold, cross_val_score     #K_FOLD CROSS VALIDATION 
k_fold = KFold(len(Y_train), n_folds=15, shuffle=True, random_state=0) #15 Folds are intorduced
score  = cross_val_score(classifier,X_encoded_train_new,Y_train, cv=k_fold, n_jobs=1)
print score
print np.mean(score)


[ 0.92473118  0.88172043  0.88172043  0.87096774  0.83870968  0.8172043
  0.78494624  0.88172043  0.89247312  0.89130435  0.85869565  0.91304348
  0.89130435  0.91304348  0.88043478]
0.874801309023


In [18]:
score

array([ 0.92473118,  0.88172043,  0.88172043,  0.87096774,  0.83870968,
        0.8172043 ,  0.78494624,  0.88172043,  0.89247312,  0.89130435,
        0.85869565,  0.91304348,  0.89130435,  0.91304348,  0.88043478])

In [ ]:
Predictions = classifier.predict(X_encoded_test_new)

In [19]:
df1 = pandas.DataFrame(score)

In [20]:
from pandas import ExcelWriter
writer = ExcelWriter('validation1.xlsx')
df1.to_excel(writer,'Sheet1')
writer.save()

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
classifier  = GradientBoostingClassifier(n_estimators=241,loss='exponential',learning_rate=0.30000000000000004,max_depth=1) 
classifier.fit(X_train_new,y_train)

GradientBoostingClassifier(init=None, learning_rate=0.3, loss='exponential',
              max_depth=1, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=241,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [22]:
y_pred = classifier.predict(X_test_new )

In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.91      0.97      0.94       307
          1       0.53      0.24      0.33        41

avg / total       0.86      0.89      0.87       348



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
X_train = X_encoded_train_new
X_test = X_encoded_train_new[:100]
y_train = Y_train
y_test = Y_train[:100]
# Fit classifier with out-of-bag estimates
params = {'n_estimators': 500, 'max_depth': 3, 'subsample': 0.5,
          'learning_rate': 0.01, 'min_samples_leaf': 1, 'random_state': 3}
clf = ensemble.GradientBoostingClassifier(**params)

clf.fit(X_train,y_train)
n_estimators = params['n_estimators']
x = np.arange(n_estimators) + 1
def heldout_score(clf, X_test, y_test):
    """compute deviance scores on ``X_test`` and ``y_test``. """
    score = np.zeros((n_estimators,), dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        score[i] = clf.loss_(y_test, y_pred)
    return score
def cv_estimate(n_folds=3):
    cv = KFold(n=X_train.shape[0], n_folds=n_folds)
    cv_clf = ensemble.GradientBoostingClassifier(**params)
    val_scores = np.zeros((n_estimators,), dtype=np.float64)
    for train, test in cv:
        cv_clf.fit(X_train[train], y_train[train])
        val_scores += heldout_score(cv_clf, X_train[test], y_train[test])
    val_scores /= n_folds
    return val_scores
cv_score = cv_estimate(3)
test_score = heldout_score(clf, X_test, y_test)
cumsum = -np.cumsum(clf.oob_improvement_)
oob_best_iter = x[np.argmin(cumsum)]
test_score -= test_score[0]
test_best_iter = x[np.argmin(test_score)]
cv_score -= cv_score[0]
cv_best_iter = x[np.argmin(cv_score)]


In [ ]:
len(y_test)

In [ ]:
cv_best_iter

In [ ]:
# plot curves and vertical lines for best iterations
oob_color = list(map(lambda x: x / 256.0, (190, 174, 212)))
test_color = list(map(lambda x: x / 256.0, (127, 201, 127)))
cv_color = list(map(lambda x: x / 256.0, (253, 192, 134)))
plt.plot(x, cumsum, label='OOB loss', color=oob_color)
plt.plot(x, test_score, label='Test loss', color=test_color)
plt.plot(x, cv_score, label='CV loss', color=cv_color)
plt.axvline(x=oob_best_iter, color=oob_color)
plt.axvline(x=test_best_iter, color=test_color)
plt.axvline(x=cv_best_iter, color=cv_color)

# add three vertical lines to xticks
xticks = plt.xticks()
xticks_pos = np.array(xticks[0].tolist() +
                      [oob_best_iter, cv_best_iter, test_best_iter])
xticks_label = np.array(list(map(lambda t: int(t), xticks[0])) +
                        ['OOB', 'CV', 'Test'])
ind = np.argsort(xticks_pos)
xticks_pos = xticks_pos[ind]
xticks_label = xticks_label[ind]
plt.xticks(xticks_pos, xticks_label)

plt.legend(loc='upper right')
plt.ylabel('normalized loss')
plt.xlabel('number of iterations')
plt.show()

In [ ]:
from __future__ import print_function

from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

# Set the parameters by cross-validation
tuned_parameters = [{'loss': ['deviance','exponential'],'max_depth': [1,2,3,4], 'learning_rate' : np.arange(0.1,0.5,0.1) }]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    gbm = GradientBoostingClassifier(n_estimators=223)
    clf = GridSearchCV(gbm, tuned_parameters, cv=5,
                       scoring='%s_weighted' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import datasets
original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                   'min_samples_split': 5}

plt.figure()

for label, color, setting in [('No shrinkage', 'orange',
                               {'learning_rate': 1.0, 'subsample': 1.0}),
                              ('learning_rate=0.1', 'turquoise',
                               {'learning_rate': 0.1, 'subsample': 1.0}),
                              ('subsample=0.5', 'blue',
                               {'learning_rate': 1.0, 'subsample': 0.5}),
                              ('learning_rate=0.1, subsample=0.5', 'gray',
                               {'learning_rate': 0.1, 'subsample': 0.5}),
                              ('learning_rate=0.1, max_features=2', 'magenta',
                               {'learning_rate': 0.1, 'max_features': 2})]:
    params = dict(original_params)
    params.update(setting)

    clf = ensemble.GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)

    # compute test set deviance
    test_deviance = np.zeros((params['n_estimators'],), dtype=np.float64)

    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        # clf.loss_ assumes that y_test[i] in {0, 1}
        test_deviance[i] = clf.loss_(y_test, y_pred)

    plt.plot((np.arange(test_deviance.shape[0]) + 1)[::5], test_deviance[::5],
            '-', color=color, label=label)

plt.legend(loc='upper left')
plt.xlabel('Boosting Iterations')
plt.ylabel('Test Set Deviance')

plt.show()

In [5]:
import pandas
import numpy as np
from sklearn.feature_extraction import DictVectorizer 
from sklearn.ensemble import GradientBoostingClassifier
#DATA PREPROCESSING 
#Storing the whole data in two dataframes 
URL = 'Health_care_Dataset_for_probelm.xlsx'
df_training = pandas.read_excel(URL,sheetname=1)
df_testing = pandas.read_excel(URL, sheetname=2)
#Copying the data-frames into other dataframes
df_train = df_training[:]
df_test = df_testing[:]

In [6]:
Y_train = df_train['Lung_Cancer'].values
del df_train['Lung_Cancer']
del df_test['Lung_Cancer']
del df_train['Patient_ID']
del df_test['Patient_ID']
# making the categorical variables in the data set as categorical variables instead of continous. 
for i in list(df_train.columns.values):
    if len(np.unique(df_train[i].values)) < 5:
        df_train[i] = df_train[i].astype(str)

for i in list(df_test.columns.values):
    if len(np.unique(df_test[i].values)) < 5:
        df_test[i] = df_test[i].astype(str)

C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
#ENCODING
X_Tr = df_train.to_dict('records')
X_Te = df_test.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_Tr)
X_te.extend(X_Te)
X_total = X_tr + X_te
#ONE-HOT-ENCODING 
enc = DictVectorizer() 
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train =X_encoded_total[:len(X_tr)]
X_train = X_encoded_train[:int(0.75*len(X_encoded_train.toarray()))]
X_test = X_encoded_train[int(0.75*len(X_encoded_train.toarray())):]
y_train = Y_train[:int(0.75*len(X_encoded_train.toarray()))]
y_test  = Y_train[int(0.75*len(X_encoded_train.toarray())):]
X_encoded_test =X_encoded_total[len(X_tr):]

In [9]:
from sklearn.feature_selection import SelectFromModel
estimator = GradientBoostingClassifier()
estimator.fit(X_encoded_train.toarray(),Y_train)
model = SelectFromModel(estimator, prefit=True)
X_encoded_train_new = model.transform(X_encoded_train.toarray())
X_encoded_test_new = model.transform(X_encoded_test.toarray())
X_train_new = model.transform(X_train.toarray())
X_test_new = model.transform(X_test.toarray())

In [15]:
classifier  = GradientBoostingClassifier(n_estimators=241,loss='exponential',learning_rate=0.30000000000000004,max_depth=1) 
classifier.fit(X_encoded_train_new,Y_train)

GradientBoostingClassifier(init=None, learning_rate=0.3, loss='exponential',
              max_depth=1, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=241,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [16]:
from sklearn.cross_validation import KFold, cross_val_score     #K_FOLD CROSS VALIDATION 
k_fold = KFold(len(Y_train), n_folds=15, shuffle=True, random_state=0) #15 Folds are intorduced
score  = cross_val_score(classifier,X_encoded_train_new,Y_train, cv=k_fold, n_jobs= 1)
print score
print np.mean(score)
y_pred = classifier.predict(X_test_new )
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[ 0.92473118  0.88172043  0.88172043  0.86021505  0.83870968  0.8172043
  0.78494624  0.86021505  0.87096774  0.89130435  0.85869565  0.91304348
  0.89130435  0.90217391  0.88043478]
0.870492441951
             precision    recall  f1-score   support

          0       0.93      1.00      0.96       307
          1       0.95      0.44      0.60        41

avg / total       0.93      0.93      0.92       348



In [17]:
Predictions = classifier.predict(X_encoded_test_new)
df = pandas.DataFrame(Predictions)
from pandas import ExcelWriter
writer = ExcelWriter('Solution2.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [19]:
df1 = pandas.DataFrame(score)
from pandas import ExcelWriter
writer = ExcelWriter('kfold2.xlsx')
df1.to_excel(writer,'Sheet1')
writer.save()

In [52]:
def Feature_Imporatance(Number):      #Number of most important features we want to look at 
            from sklearn.ensemble import GradientBoostingClassifier
            from operator import itemgetter
            from collections import OrderedDict
            import matplotlib.pyplot as plt
            df_t = df_training[:]
            Y = df_t['Lung_Cancer'].values
            del df_t['Lung_Cancer']
            del df_t['Patient_ID']
            X_dup = df_t.to_dict('records')
            X_traing = []
            X_traing.extend(X_dup)
            X = enc.fit_transform(X_traing)
            classification = GradientBoostingClassifier()
            classification.fit(X,Y)
            Importance = classification.feature_importances_*100
            Headings = sorted(X_Te[0].keys())
            FeatureImportance =  dict(zip( Headings,Importance))
            FeatureImportanceSorted = (sorted(FeatureImportance.items(), key=lambda t: t[1]))
            Headings_sorted =[]
            Importance_sorted = []
            for i in range(len(FeatureImportanceSorted)):
                Headings_sorted.append(FeatureImportanceSorted[i][0])
                Importance_sorted.append(FeatureImportanceSorted[i][1])
            Headings_sorted = list(Headings_sorted[::-1])
            Importance_sorted = list(Importance_sorted[::-1])
            labels = Headings_sorted[:Number]
            labels.append('other')
            slices = Importance_sorted[:Number]
            slices.append(sum(Importance_sorted[Number:]))
            global classification  
            global Headings_sorted
            global Importance_sorted  
Feature_Imporatance(10)           


<ipython-input-52-1a7e40b6ba52>:31: SyntaxWarning: name 'classification' is assigned to before global declaration
  global classification
<ipython-input-52-1a7e40b6ba52>:32: SyntaxWarning: name 'Headings_sorted' is assigned to before global declaration
  global Headings_sorted
<ipython-input-52-1a7e40b6ba52>:33: SyntaxWarning: name 'Importance_sorted' is assigned to before global declaration
  global Importance_sorted


In [55]:
import pandas
import matplotlib.pyplot as plt
from pandas.tools.plotting import parallel_coordinates
data = df_training
a = Headings_sorted[:10]
a.append('Lung_Cancer')
data = data[a]
parallel_coordinates(data,'Lung_Cancer')
plt.show()

# Final Code 

In [26]:
import pandas
import numpy as np
from sklearn.feature_extraction import DictVectorizer 
from sklearn.ensemble import GradientBoostingClassifier
#DATA PREPROCESSING 
#Storing the whole data in two dataframes 
URL = 'Health_care_Dataset_for_probelm.xlsx'
#df_training = pandas.read_excel(URL,sheetname=1)
#df_testing = pandas.read_excel(URL, sheetname=2)
#Copying the data-frames into other dataframes
df_train = df_training[:]
df_test = df_testing[:]  
#Deleting the Patient_ID column as it doesnot make any significance in model building 
Y_train = df_train['Lung_Cancer'].values
del df_train['Lung_Cancer']
del df_test['Lung_Cancer']
del df_train['Patient_ID']
del df_test['Patient_ID']
# making the categorical variables in the data set as categorical variables instead of continous. 
for i in list(df_train.columns.values):
    if len(np.unique(df_train[i].values)) < 5:
        df_train[i] = df_train[i].astype(str)

for i in list(df_test.columns.values):
    if len(np.unique(df_test[i].values)) < 5:
        df_test[i] = df_test[i].astype(str)
#ENCODING
X_Tr = df_train.to_dict('records')
X_Te = df_test.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_Tr)
X_te.extend(X_Te)
X_total = X_tr + X_te
#ONE-HOT-ENCODING 
enc = DictVectorizer() 
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train =X_encoded_total[:len(X_tr)]
X_train = X_encoded_train[:int(0.75*len(X_encoded_train.toarray()))]
X_test = X_encoded_train[int(0.75*len(X_encoded_train.toarray())):]
y_train = Y_train[:int(0.75*len(X_encoded_train.toarray()))]
y_test  = Y_train[int(0.75*len(X_encoded_train.toarray())):]
X_encoded_test =X_encoded_total[len(X_tr):]
#END OF DATA PREPROCESSING- 
######################################################
#FEATURE ENGINEERING 
#UNDERSTANDING FEATURE IMPORTANCE
#Initially we will run a Gradient-Boosting Trees(Basic) to Understand the Importance of each Factor(UN-Categorized data)
def Feature_Imporatance(Number):      #Number of most important features we want to look at 
            from sklearn.ensemble import GradientBoostingClassifier
            from operator import itemgetter
            from collections import OrderedDict
            import matplotlib.pyplot as plt
            df_t = df_training[:]
            Y = df_t['Lung_Cancer'].values
            del df_t['Lung_Cancer']
            X_dup = df_t.to_dict('records')
            X_traing = []
            X_traing.extend(X_dup)
            X = enc.fit_transform(X_traing)
            classification = GradientBoostingClassifier()
            classification.fit(X,Y)
            Importance = classification.feature_importances_*100
            Headings = sorted(X_Te[0].keys())
            FeatureImportance =  dict(zip( Headings,Importance))
            FeatureImportanceSorted = (sorted(FeatureImportance.items(), key=lambda t: t[1]))
            Headings_sorted =[]
            Importance_sorted = []
            for i in range(len(FeatureImportanceSorted)):
                Headings_sorted.append(FeatureImportanceSorted[i][0])
                Importance_sorted.append(FeatureImportanceSorted[i][1])
            Headings_sorted = list(Headings_sorted[::-1])
            Importance_sorted = list(Importance_sorted[::-1])
            labels = Headings_sorted[:Number]
            labels.append('other')
            slices = Importance_sorted[:Number]
            slices.append(sum(Importance_sorted[Number:]))
            #Plotting The Values
            explode = (0.0, 0, 0, 0.1,0,0,0,0) 
            plt.pie(slices, labels=labels,
                    autopct='%1.1f%%', shadow=False, startangle=90,labeldistance=1.2, ) 
            plt.axis('equal')
            #plt.show() 
            global classification  
            global Headings_sorted
            global Importance_sorted        
#FEATURE REDUCTION 
#Initially we will run a Gradient-Boosting Trees(Basic) to reduce the data set in terms of Number of features 
from sklearn.feature_selection import SelectFromModel
estimator = GradientBoostingClassifier()
estimator.fit(X_encoded_train.toarray(),Y_train)
model = SelectFromModel(estimator, prefit=True)
X_encoded_train_new = model.transform(X_encoded_train.toarray())
X_encoded_test_new = model.transform(X_encoded_test.toarray())
X_train_new = model.transform(X_train.toarray())
X_test_new = model.transform(X_test.toarray())
#END FEATURE ENGINEERING 
#################################################################################
#MODEL BUILDING
#HYPER PARAMETRIC TUNING
def N_estimators(X_train,X_test,y_train,y_test):    #Running a Cross validation Iterations for finding the best values for N-estimators
            import numpy as np
            import matplotlib.pyplot as plt
            from sklearn import ensemble
            from sklearn.cross_validation import KFold
            from sklearn.cross_validation import train_test_split
            X_train = X_encoded_train_new
            X_test = X_encoded_train_new[:100]
            y_train = Y_train
            y_test = Y_train
            # Fit classifier with out-of-bag estimates
            params = {'n_estimators': 500, 'max_depth': 3, 'subsample': 0.5,
                      'learning_rate': 0.01, 'min_samples_leaf': 1, 'random_state': 3}
            clf = ensemble.GradientBoostingClassifier(**params)

            clf.fit(X_train,y_train)
            n_estimators = params['n_estimators']
            x = np.arange(n_estimators) + 1
            def heldout_score(clf, X_test, y_test):
                """compute deviance scores on ``X_test`` and ``y_test``. """
                score = np.zeros((n_estimators,), dtype=np.float64)
                for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
                    score[i] = clf.loss_(y_test, y_pred)
                return score
            def cv_estimate(n_folds=3):
                cv = KFold(n=X_train.shape[0], n_folds=n_folds)
                cv_clf = ensemble.GradientBoostingClassifier(**params)
                val_scores = np.zeros((n_estimators,), dtype=np.float64)
                for train, test in cv:
                    cv_clf.fit(X_train[train], y_train[train])
                    val_scores += heldout_score(cv_clf, X_train[test], y_train[test])
                val_scores /= n_folds
                return val_scores
            cv_score = cv_estimate(3)
            test_score = heldout_score(clf, X_test, y_test)
            cumsum = -np.cumsum(clf.oob_improvement_)
            oob_best_iter = x[np.argmin(cumsum)]
            test_score -= test_score[0]
            test_best_iter = x[np.argmin(test_score)]
            cv_score -= cv_score[0]
            cv_best_iter = x[np.argmin(cv_score)]
            global cv_best_iter     
#N_estimators(X_encoded_train_new, X_encoded_train_new[:100],Y_train,Y_train[:100]) #cv_best_iter gives the best N_parameter value
#cv_best_iter = 241 trees is the best parameter which minimizes the CV- loss funtion
def Other_Parameters(X_train, y_train):
            from sklearn import datasets
            from sklearn.cross_validation import train_test_split
            from sklearn.grid_search import GridSearchCV
            from sklearn.metrics import classification_report
            from sklearn.ensemble import GradientBoostingClassifier
            tuned_parameters = [{'loss': ['deviance','exponential'],'max_depth': [1,2,3,4], 'learning_rate' : np.arange(0.1,0.5,0.1) }]
            scores = ['precision', 'recall']
            for score in scores:
                print("%s" % score)
                gbm = GradientBoostingClassifier(n_estimators=232)
                clf = GridSearchCV(gbm, tuned_parameters, cv=5,
                                   scoring='%s_weighted' % score)
                clf.fit(X_train, y_train)
                print (clf.best_params_)
#Other_Parameters(X_encoded_train_new,Y_train)
#{'loss': 'exponential', 'learning_rate': 0.30000000000000004, 'max_depth': 1}   are best parameters after running the above funtion          
#END HYPER PARAMETRIC TUNING 
################################################
#MODEL TRAINING 
#all parameters are comuputed to maximize the goodness of the fit
classifier  = GradientBoostingClassifier(n_estimators=241,loss='exponential',learning_rate=0.30000000000000004,max_depth=1) 
classifier.fit(X_encoded_train_new,Y_train)
#END MODEL TRAINING 
##################################################
#CROSS VALIDATING 
from sklearn.cross_validation import KFold, cross_val_score     #K_FOLD CROSS VALIDATION 
k_fold = KFold(len(Y_train), n_folds=15, shuffle=True, random_state=0) #15 Folds are intorduced
score  = cross_val_score(classifier,X_encoded_train_new,Y_train, cv=k_fold, n_jobs= 1)
print score
print np.mean(score)
y_pred = classifier.predict(X_test_new )
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
#END CROSS VALIDATING 
################################################
#RUNNING ON THE TEST SET 
Predictions = classifier.predict(X_encoded_test_new)
df = pandas.DataFrame(Predictions)
from pandas import ExcelWriter
writer = ExcelWriter('Solution.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()
#Visualization 
import pandas
import matplotlib.pyplot as plt
from pandas.tools.plotting import parallel_coordinates
data = df_training
a = Headings_sorted[:10]
a.append('Lung_Cancer')
data = data[a]
parallel_coordinates(data,'Lung_Cancer')
plt.show()
#End Visualization


C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
<ipython-input-26-7720df067f5f>:84: SyntaxWarning: name 'classification' is assigned to before global declaration
  global classification
<ipython-input-26-7720df067f5f>:85: SyntaxWarning: name 'Headings_sorted' is assigned to before global declaration
  global He

[ 0.92473118  0.89247312  0.88172043  0.86021505  0.83870968  0.8172043
  0.77419355  0.87096774  0.88172043  0.89130435  0.85869565  0.90217391
  0.89130435  0.90217391  0.88043478]
0.871201496026
             precision    recall  f1-score   support

          0       0.91      0.97      0.94       307
          1       0.53      0.24      0.33        41

avg / total       0.86      0.89      0.87       348



NameError: name 'Headings_sorted' is not defined

In [ ]:
def N_estimators(X_train,X_test,y_train,y_test):    #Running a Cross validation Iterations for finding the best values for N-estimators
            import numpy as np
            import matplotlib.pyplot as plt
            from sklearn import ensemble
            from sklearn.cross_validation import KFold
            from sklearn.cross_validation import train_test_split
            # Fit classifier with out-of-bag estimates
            params = {'n_estimators': 500, 'max_depth': 3, 'subsample': 0.5,
                      'learning_rate': 0.01, 'min_samples_leaf': 1, 'random_state': 3}
            clf = ensemble.GradientBoostingClassifier(**params)

            clf.fit(X_train,y_train)
            n_estimators = params['n_estimators']
            x = np.arange(n_estimators) + 1
            def heldout_score(clf, X_test, y_test):
                """compute deviance scores on ``X_test`` and ``y_test``. """
                score = np.zeros((n_estimators,), dtype=np.float64)
                for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
                    score[i] = clf.loss_(y_test, y_pred)
                return score
            def cv_estimate(n_folds=3):
                cv = KFold(n=X_train.shape[0], n_folds=n_folds)
                cv_clf = ensemble.GradientBoostingClassifier(**params)
                val_scores = np.zeros((n_estimators,), dtype=np.float64)
                for train, test in cv:
                    cv_clf.fit(X_train[train], y_train[train])
                    val_scores += heldout_score(cv_clf, X_train[test], y_train[test])
                val_scores /= n_folds
                return val_scores
            cv_score = cv_estimate(3)
            test_score = heldout_score(clf, X_test, y_test)
            cumsum = -np.cumsum(clf.oob_improvement_)
            oob_best_iter = x[np.argmin(cumsum)]
            test_score -= test_score[0]
            test_best_iter = x[np.argmin(test_score)]
            cv_score -= cv_score[0]
            cv_best_iter = x[np.argmin(cv_score)]
            global cv_best_iter

In [ ]:
N_estimators(X_encoded_train_new, X_encoded_train_new[:100],Y_train,Y_train[:100])


In [ ]:
def Other_Parameters(X_train, y_train):
            from sklearn import datasets
            from sklearn.cross_validation import train_test_split
            from sklearn.grid_search import GridSearchCV
            from sklearn.metrics import classification_report
            from sklearn.ensemble import GradientBoostingClassifier

            # Set the parameters by cross-validation
            tuned_parameters = [{'loss': ['deviance','exponential'],'max_depth': [1,2,3,4], 'learning_rate' : np.arange(0.1,0.5,0.1) }]

            scores = ['precision', 'recall']

            for score in scores:
                print("%s" % score)
                print()
                gbm = GradientBoostingClassifier(n_estimators=232)
                clf = GridSearchCV(gbm, tuned_parameters, cv=5,
                                   scoring='%s_weighted' % score)
                clf.fit(X_train, y_train)
                print(clf.best_params_)

In [ ]:
Other_Parameters(X_encoded_train_new,Y_train)

In [ ]:
classifier  = GradientBoostingClassifier(n_estimators=232,loss='exponential',learning_rate=0.30000000000000004,max_depth=1)

In [ ]:
classifier.fit(X_encoded_train_new,Y_train)

In [ ]:
from sklearn.cross_validation import KFold, cross_val_score
k_fold = KFold(len(Y_train), n_folds=15, shuffle=True, random_state=0)
cross_val_score(classifier,X_encoded_train_new,Y_train, cv=k_fold, n_jobs=-1)

In [ ]:
def Feature_Imporatance(Number):      #Number of most important features we want to look at 
            from sklearn.ensemble import GradientBoostingClassifier
            from operator import itemgetter
            from collections import OrderedDict
            import matplotlib.pyplot as plt
            df_t = df_training[:]
            Y = df_t['Lung_Cancer'].values
            del df_t['Lung_Cancer']
            X_dup = df_t.to_dict('records')
            X_traing = []
            X_traing.extend(X_dup)
            X = enc.fit_transform(X_traing)
            classification = GradientBoostingClassifier()
            classification.fit(X,Y)
            Importance = classification.feature_importances_*100
            Headings = sorted(X_Te[0].keys())
            FeatureImportance =  dict(zip( Headings,Importance))
            FeatureImportanceSorted = (sorted(FeatureImportance.items(), key=lambda t: t[1]))
            Headings_sorted =[]
            Importance_sorted = []
            for i in range(len(FeatureImportanceSorted)):
                Headings_sorted.append(FeatureImportanceSorted[i][0])
                Importance_sorted.append(FeatureImportanceSorted[i][1])
            Headings_sorted = list(Headings_sorted[::-1])
            Importance_sorted = list(Importance_sorted[::-1])
            labels = Headings_sorted[:Number]
            labels.append('other')
            slices = Importance_sorted[:Number]
            slices.append(sum(Importance_sorted[Number:]))
            #Plotting The Values
            explode = (0.0, 0, 0, 0.1,0,0,0,0) 
            plt.pie(slices, labels=labels,
                    autopct='%1.1f%%', shadow=False, startangle=90,labeldistance=1.2, ) 
            plt.axis('equal')
            #plt.show() 
            global classification  
            global Headings_sorted
            global Importance_sorted        

In [ ]:
Headings_sorted

In [5]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm

fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y, Z = axes3d.get_test_data(0.05)
ax.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3)
cset = ax.contourf(X, Y, Z, zdir='z', offset=-100, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Z, zdir='x', offset=-40, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Z, zdir='y', offset=40, cmap=cm.coolwarm)

ax.set_xlabel('X')
ax.set_xlim(-40, 40)
ax.set_ylabel('Y')
ax.set_ylim(-40, 40)
ax.set_zlabel('Z')
ax.set_zlim(-100, 100)

plt.show()

In [82]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np

n_angles = 36
n_radii = 8

# An array of radii
# Does not include radius r=0, this is to eliminate duplicate points
radii = np.linspace(0.125, 1.0, n_radii)

# An array of angles
angles = np.linspace(0, 2*np.pi, n_angles, endpoint=False)

# Repeat all angles for each radius
angles = np.repeat(angles[..., np.newaxis], n_radii, axis=1)

# Convert polar (radii, angles) coords to cartesian (x, y) coords
# (0, 0) is added here. There are no duplicate points in the (x, y) plane
x = ma['T'].values
y = ma['Strain rate'].values
X, Y = np.meshgrid(x, y)
Z = ma['Efficiency'].values
V = ma['Instability'].values

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot_trisurf(x, y, Z, cmap=cm.jet(V), linewidth=0.2)

plt.show()

TypeError: unhashable type: 'numpy.ndarray'

In [26]:
cm.jet

In [25]:
co = np.array(len(x))

In [19]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm

fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y, Z = axes3d.get_test_data(0.05)
ax.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3)
cset = ax.contourf(X, Y, Z, zdir='z', offset=-100, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Z, zdir='x', offset=-40, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Z, zdir='y', offset=40, cmap=cm.coolwarm)

ax.set_xlabel('X')
ax.set_xlim(-40, 40)
ax.set_ylabel('Y')
ax.set_ylim(-40, 40)
ax.set_zlabel('Z')
ax.set_zlim(-100, 100)

plt.show()


In [32]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
 
x = np.linspace(-8, 8, 100)
y = np.linspace(-8, 8, 100)
X, Y = np.meshgrid(x, y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)/R
 
fig = plt.figure()
ax = fig.gca(projection = '3d')
 
surf = ax.plot_surface(X, Y, Z,
                      rstride = 3,
                      cstride = 3,
                      cmap = cm.coolwarm,
                      linewidth = 0.5,
                      antialiased = True)
 
ax.view_init(elev=60, azim=50)
ax.dist=8 
plt.show()

In [158]:
ma= pd.read_csv('harshit_MS.csv')

In [159]:
ma

,T,Strain rate,Efficiency,Instability
0,800,0.1,0.27180,0.06542
1,800,1.0,0.20030,-0.06190
2,800,10.0,0.12240,-0.03344
3,900,0.1,0.35250,3.25005
4,900,1.0,0.44970,0.14065
5,900,10.0,0.28000,-2.96870
6,1000,0.1,0.12220,0.50169
7,1000,1.0,0.27169,0.41420
8,1000,10.0,0.39910,0.32669


In [160]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


x = ma['Strain rate'].values
y = ma['T'].values
X, Y = np.meshgrid(x, y)
Z,Z = np.meshgrid(ma['Efficiency'].values,ma['Efficiency'].values)
V,V = np.meshgrid(ma['Instability'].values,ma['Instability'].values)

# create the figure, add a 3d axis, set the viewing angle
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.view_init(45,60)

# here we create the surface plot, but pass V through a colormap
# to create a different color for each patch
ax.plot_surface(X, Y,Z, facecolors=cm.jet(V))
plt.show()

In [136]:
Z

array([[-0.06185, -0.06185, -0.06185, -0.06185, -0.06185, -0.06185,
        -0.06185, -0.06185, -0.06185, -0.06185, -0.06185, -0.06185],
       [-0.01405, -0.01405, -0.01405, -0.01405, -0.01405, -0.01405,
        -0.01405, -0.01405, -0.01405, -0.01405, -0.01405, -0.01405],
       [ 0.03157,  0.03157,  0.03157,  0.03157,  0.03157,  0.03157,
         0.03157,  0.03157,  0.03157,  0.03157,  0.03157,  0.03157],
       [ 0.27745,  0.27745,  0.27745,  0.27745,  0.27745,  0.27745,
         0.27745,  0.27745,  0.27745,  0.27745,  0.27745,  0.27745],
       [ 0.18008,  0.18008,  0.18008,  0.18008,  0.18008,  0.18008,
         0.18008,  0.18008,  0.18008,  0.18008,  0.18008,  0.18008],
       [ 0.07098,  0.07098,  0.07098,  0.07098,  0.07098,  0.07098,
         0.07098,  0.07098,  0.07098,  0.07098,  0.07098,  0.07098],
       [ 0.18095,  0.18095,  0.18095,  0.18095,  0.18095,  0.18095,
         0.18095,  0.18095,  0.18095,  0.18095,  0.18095,  0.18095],
       [ 0.26539,  0.26539,  0.26539,  0.

In [142]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# create some fake data
x = y = np.arange(12)
# here are the x,y and respective z values
X, Y = np.meshgrid(x, y)
Z = np.sinc(np.sqrt(X*X+Y*Y))
# this is the value to use for the color
V = np.sin(Y)

# create the figure, add a 3d axis, set the viewing angle
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(45,60)

# here we create the surface plot, but pass V through a colormap
# to create a different color for each patch
ax.plot_surface(X, Y, Z, facecolors=cm.Oranges(V))
plt.show()

IndexError: index 10 is out of bounds for axis 0 with size 4

In [144]:
len(ma['Instability'].values)

12

In [145]:
np.sin(Y)

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.84147098,  0.84147098,  0.84147098,  0.84147098,  0.84147098,
         0.84147098,  0.84147098,  0.84147098,  0.84147098,  0.84147098,
         0.84147098,  0.84147098],
       [ 0.90929743,  0.90929743,  0.90929743,  0.90929743,  0.90929743,
         0.90929743,  0.90929743,  0.90929743,  0.90929743,  0.90929743,
         0.90929743,  0.90929743],
       [ 0.14112001,  0.14112001,  0.14112001,  0.14112001,  0.14112001,
         0.14112001,  0.14112001,  0.14112001,  0.14112001,  0.14112001,
         0.14112001,  0.14112001],
       [-0.7568025 , -0.7568025 , -0.7568025 , -0.7568025 , -0.7568025 ,
        -0.7568025 , -0.7568025 , -0.7568025 , -0.7568025 , -0.7568025 ,
        -0.7568025 , -0.7568025 ],
       [-0.95892427, -0.95892427, -0.95892427, -0.95892427, -0.95892427,
        -0.95892427, -